In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

test_task_for_ds_time_series_forecasting_2024_10_path = kagglehub.competition_download('test-task-for-ds-time-series-forecasting-2024-10')

print('Data source import complete.')


In [ ]:
#Завантаження бібліотек

import pandas as pd
import numpy as np
import holidays
import statsmodels.api as sm
from prophet import Prophet
import optuna
from sklearn.metrics import mean_absolute_error, mean_squared_error, median_absolute_error

In [ ]:
#Завантаження датасету

csv_path = '/kaggle/input/test-task-for-ds-time-series-forecasting-2024-10/ts_hist.csv'
df = pd.read_csv(csv_path)
data = df.copy()

In [ ]:
# Перегляд даних в датаседі

data.head()

In [ ]:
#Перевірка наявності порожніх значень

data.isna().sum()

In [ ]:
# Заповнення NaN пустими рядками

data = data.fillna("")

In [ ]:
# Створення переліку унікальних кодів товарів ('index')

unique_indexes = data['index'].unique()

In [ ]:
# Перевірка наявності для кожного продукту ('index') всіх дат
# в проміжку між потатковою датою та '2016-05-15'
# '2016-05-15' - дата, станом на яку надано інформацію
# прогноз буде проводитись на тиждень з 2016-05-16 по 2016-05-22

# Створення порожнього DataFrame для зберігання загальних результатів
results = []

for product in unique_indexes:
    # Фільтрування даних окремо для кожного товару
    product_data = data[data['index'] == product]
    product_data = product_data.sort_values('date')

    product_data = product_data.drop(columns=['index', 'store_id', 'cat_id', 'dept_id','item_id',
                                              'event_name_1','event_type_1','event_name_2','event_type_2'])


    # Перетворення значення в формат дати
    product_data['date'] = pd.to_datetime(product_data['date'])

    # Встановлення date як індексу
    product_data.set_index('date', inplace=True)

    # Цільова дата
    target_date = pd.Timestamp('2016-05-15')

    # Створення нового індексу, який включатиме всі дати до цільової дати
    new_index = pd.date_range(start=product_data.index.min(),
                              end=target_date,
                              freq='D',
                              name='date')

    # Переіндексування DataFrame
    product_data = product_data.reindex(new_index)

    # Заповнення пропуски в колонці 'qnt' нулями
    product_data['qnt'] = product_data['qnt'].fillna(value=0)
    product_data['index'] = product

    # Виконання декомпозиції
    try:
        decomp = sm.tsa.seasonal_decompose(product_data['qnt'], model='additive')


        # Додавання результату до списку
        results.append(product_data)

    except Exception as e:
        print(f"Error processing {product}: {e}")

# Об'єднання всіх результатів
data = pd.concat(results)
data['qnt'] = data['qnt'].round().astype(int)

data

In [ ]:
#Попередня обробка даних

# Перетворення date у формат datetime
data = data.reset_index()
data['date'] = pd.to_datetime(data['date'])

# Створення вітмітки чи святковий день
uk_holidays = holidays.UK()
data['is_holiday'] = data['date'].apply(lambda x: 1 if x in uk_holidays else 0)

# Агрегація даних до тижневого рівня з урахуванням 'index'
weekly_data = data.groupby(['index', pd.Grouper(key='date', freq='W-Mon')])['qnt'].sum().reset_index()

# Ступеневе перетворення для стабілізації дисперсії
weekly_data['log_qnt'] = np.sqrt(weekly_data['qnt'])

# Створення додаткових ознак лагів
weekly_data['lag_1'] = weekly_data.groupby('index')['log_qnt'].shift(1)
weekly_data['lag_2'] = weekly_data.groupby('index')['log_qnt'].shift(2)
weekly_data['lag_3'] = weekly_data.groupby('index')['log_qnt'].shift(3)
weekly_data['lag_4'] = weekly_data.groupby('index')['log_qnt'].shift(4)
weekly_data['lag_5'] = weekly_data.groupby('index')['log_qnt'].shift(5)
weekly_data['lag_6'] = weekly_data.groupby('index')['log_qnt'].shift(6)
weekly_data['lag_7'] = weekly_data.groupby('index')['log_qnt'].shift(7)
weekly_data['lag_8'] = weekly_data.groupby('index')['log_qnt'].shift(8)

# Заміна NaN значення в лагах нулями
weekly_data.fillna(0, inplace=True)


In [ ]:
#Обрабка категоріальних змін

weekly_data['category'] = weekly_data['index'].apply(lambda x: 0 if 'FOODS' in x else
                                                               (2 if 'HOBBIES' in x else
                                                               (1 if 'HOUSEHOLD' in x else None)))

weekly_data['index_split'] = weekly_data['index'].str.split('_')
weekly_data['store_id'] = weekly_data['index_split'].apply(lambda x: x[0])
weekly_data['dept_id'] = weekly_data['index_split'].apply(lambda x: x[2])
weekly_data['item_id'] = weekly_data['index_split'].apply(lambda x: x[3])
weekly_data = weekly_data.drop(columns=['index_split'])

weekly_data['dept_id'] = weekly_data['dept_id'].astype(int)
weekly_data['store_id'] = weekly_data['store_id'].astype(int)
weekly_data['item_id'] = weekly_data['item_id'].astype(int)

In [ ]:
print(weekly_data.dtypes)

In [ ]:
#Створення датасету з переліком тижнів, у яких є святкові дні

holidays_data = data.groupby(['index', pd.Grouper(key='date', freq='W-Mon')])['is_holiday'].sum().reset_index()

# Фільтрація записів, де дні відмічені як святкові
holidays_data = holidays_data.query('is_holiday == 1')

# Створення колонки 'holiday' і заповнення значенням 'holiday'
# (для по дальшого використання в моделі)

holidays_data['holiday'] = 'holiday'
holidays_data = holidays_data[['date', 'holiday']]
holidays_data = holidays_data.rename(columns={'date': 'ds'})

# Вивід результату
print(holidays_data)

In [ ]:
#Перевірка чи є свята в тижні, на який робиться прогноз

# Визначення переліку свят
uk_holidays = holidays.UK()

# Період для прогнозу
start_date = pd.Timestamp('2016-05-16')
end_date = pd.Timestamp('2016-05-22')
date_range = pd.date_range(start=start_date, end=end_date)

# Перевірка, чи є свята в періоді
holidays_in_period = [date for date in date_range if date in uk_holidays]

if holidays_in_period:
    print("Є свята у вказаному періоді:")
    for holiday in holidays_in_period:
        print(f"{holiday.date()} - {uk_holidays[holiday]}")
else:
    print("Свят у вказаному періоді немає.")

In [ ]:
#Для тестування на кількох товарах

#unique_indexes = data['index'].unique()
#unique_indexes = unique_indexes[49::50]
#len(unique_indexes)

In [ ]:
# Глобальний DataFrame для зберігання прогнозів
all_forecasts = pd.DataFrame()

def objective(trial, product_index, weekly_data):
    global all_forecasts  # Доступ до глобального DataFrame

    validation_data = weekly_data[weekly_data['date'] == '2016-05-16']
    train_data = weekly_data[weekly_data['date'] != '2016-05-16']

    # Оптимізація параметрів
    changepoint_prior_scale = trial.suggest_float(
            'changepoint_prior_scale', 0.001, 0.5, log=True
        ) if trial.number > 0 else 0.05

    seasonality_prior_scale = trial.suggest_float(
                'seasonality_prior_scale', 0.01, 10.0, log=True
            ) if trial.number > 0 else 1.0

    seasonality_mode = trial.suggest_categorical(
                'seasonality_mode', ['additive', 'multiplicative']
            ) if trial.number > 0 else 'additive'

    holidays_prior_scale = trial.suggest_float(
                'holidays_prior_scale', 0.01, 10.0, log=True
            ) if trial.number > 0 else 1.0

    # Підготовка даних
    subset_train_data = train_data[train_data['index'] == product_index]
    subset_validation_data = validation_data[validation_data['index'] == product_index]

    # Перевірка, чи є дані для навчання та валідації
    if subset_train_data.empty or subset_validation_data.empty:
        return float('inf')  # Якщо немає даних, повертаємо "поганий" результат

    # Створення моделі Prophet
    model = Prophet(holidays=holidays_data,
            changepoint_prior_scale=changepoint_prior_scale,
            seasonality_prior_scale=seasonality_prior_scale,
            seasonality_mode=seasonality_mode,
            holidays_prior_scale=holidays_prior_scale)

    # Додавання регресорів
    model.add_regressor('store_id')
    model.add_regressor('dept_id')
    model.add_regressor('item_id')
    model.add_regressor('category')

    # Додавання лагів
    for lag in range(1, 9):
        model.add_regressor(f'lag_{lag}')

    # Підготовка даних для моделі
    prophet_data = subset_train_data.rename(columns={'date': 'ds', 'qnt': 'y'})

    model.fit(prophet_data)

    # Прогнозування
    future_dates = model.make_future_dataframe(periods=2, freq='W-Mon')

    # Додавання додаткових ознак
    for lag in range(1, 9):
        future_dates[f'lag_{lag}'] = subset_train_data[f'lag_{lag}'].iloc[-1]

    future_dates['store_id'] = subset_train_data['store_id'].iloc[-1]
    future_dates['dept_id'] = subset_train_data['dept_id'].iloc[-1]
    future_dates['item_id'] = subset_train_data['item_id'].iloc[-1]
    future_dates['category'] = subset_train_data['category'].iloc[-1]

    forecast = model.predict(future_dates)

    # Вибір прогнозу для валідаційних даних (перший тиждень)
    validation_forecast = forecast.tail(2)  # Останні 2 рядки є прогнозом на 2 тижні
    validation_forecast = validation_forecast.reset_index(drop=True)

    # Збереження прогнозу на другий тиждень у глобальний DataFrame
    second_week_forecast = validation_forecast.iloc[1]  # Другий рядок є прогнозом на другий тиждень
    second_week_forecast_df = pd.DataFrame(second_week_forecast).T
    second_week_forecast_df['index'] = product_index
    all_forecasts = pd.concat([all_forecasts, second_week_forecast_df], ignore_index=True)

    # Вибір відповідних значень для MAE
    actual_values = subset_validation_data['qnt'].values
    predicted_values = validation_forecast['yhat'].values[:len(actual_values)]

    mae = mean_absolute_error(actual_values, predicted_values)

    # Збереження прогнозу і помилки mae у DataFrame
    validation_forecast['mae'] = mae
    validation_forecast['index'] = product_index
    return validation_forecast


def optimize_parameters_for_product(product_index, weekly_data):
    temp_forecasts = []  # Список для збереження всіх прогнозів для кожного trial

    def objective_with_save(trial):
        forecast = objective(trial, product_index, weekly_data)
        temp_forecasts.append(forecast)  # Зберігаємо прогноз у список
        return forecast['mae'].iloc[0]  # Повертаємо тільки помилку mae

    study = optuna.create_study(direction='minimize')
    study.optimize(objective_with_save, n_trials=50)

    # Отримуємо найкращий прогноз для даного продукту
    best_forecast = min(temp_forecasts, key=lambda x: x['mae'].iloc[0])

    global all_forecasts
    all_forecasts = pd.concat([all_forecasts, best_forecast], ignore_index=True)

# Виклик оптимізації для кожного продукту
for product_index in unique_indexes:
    optimize_parameters_for_product(product_index, weekly_data)

In [ ]:
#Перевірка результату
all_forecasts[['index','yhat','mae' ]]

In [ ]:
# Обробка результату
one_week_forecasts = all_forecasts[['index','yhat','mae']]
one_week_forecasts = one_week_forecasts.dropna()
one_week_forecasts

In [ ]:
#збереження середнього значення прогнозу у разі наявності однакового MAE з різним прогнозом
average_yhat = one_week_forecasts.groupby('index', as_index=False)['yhat'].mean()
average_yhat.loc[:, 'yhat'] = average_yhat['yhat'].round().astype(int)
average_yhat

In [ ]:
# Збереження результатів в CSV файл
average_yhat.to_csv('submission_aver.csv', index=False)

In [ ]:
#збереження максимального значення прогнозу у разі наявності однакового MAE з різним прогнозом
top_yhat = one_week_forecasts.sort_values(by=['index', 'yhat'], ascending=False)
top_yhat = top_yhat.drop_duplicates(subset=['index'], keep='first')

top_yhat.loc[:, 'yhat'] = top_yhat['yhat'].round().astype(int)
top_yhat

In [ ]:
# Збереження результатів в CSV файл
top_yhat[['index','yhat']].to_csv('submission_max.csv', index=False)

In [ ]:
#збереження мінімального значення прогнозу у разі наявності однакового MAE з різним прогнозом
low_yhat = one_week_forecasts.sort_values(by=['index', 'yhat'])
low_yhat = low_yhat.drop_duplicates(subset=['index'], keep='first')

low_yhat.loc[:, 'yhat'] = low_yhat['yhat'].round().astype(int)
low_yhat

In [ ]:
# Збереження результатів в CSV файл
low_yhat[['index','yhat']].to_csv('submission.csv', index=False)